In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf 
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers

import keras as keras

from keras import models
from keras import metrics
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LeakyReLU, BatchNormalization
from keras.optimizers import SGD
from keras.utils.np_utils import to_categorical
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelBinarizer
import math
from keras import backend as K

Using TensorFlow backend.


In [40]:
#tf.enable_eager_execution()

In [125]:
labels = [[0.0, 1.0, 2.0]]
#labels = [[0,1,2]]
ratio_0_white = 744.0 / 1599
ratio_1_white = 638.0 / 1599
ratio_2_white = 217.0 / 1599
ratio_0_red = 1640.0 / 4898
ratio_1_red = 2198.0 / 4898
ratio_2_red = 1060.0 / 4898
class_weight = tf.constant([[ratio_0_red, ratio_1_red, ratio_2_red]])
logits = [0] # shape [batch_size, 3]
weight_per_label = tf.transpose( tf.matmul(labels
                           , tf.transpose(class_weight)) ) #shape [1, batch_size]
#labels = [[0,1,2]]
xent = tf.multiply(weight_per_label, 
                   tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=labels, name="xent_raw")) #shape [1, batch_size]
#shape 1


TypeError: Value passed to parameter 'features' has DataType int32 not in list of allowed values: float16, bfloat16, float32, float64

In [127]:
logits = ... # shape: [batch_size, 4]
scaled_logits = tf.multiply(logits, class_weight) # shape [batch_size, 4]
loss = tf.nn.softmax_cross_entropy_with_logits(logits=scaled_logits, labels=labels)

TypeError: Failed to convert object of type <class 'ellipsis'> to Tensor. Contents: Ellipsis. Consider casting elements to a supported type.

In [2]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 

In [3]:
def confusion_matrix_report(y_true, y_pred):    
    cm, labels = confusion_matrix(y_true, y_pred), unique_labels(y_true, y_pred)
    column_width = max([len(str(x)) for x in labels] + [5])  # 5 is value length
    report = " " * column_width + " " + "{:_^{}}".format("Prediction", column_width * len(labels))+ "\n"
    report += " " * column_width + " ".join(["{:>{}}".format(label, column_width) for label in labels]) + "\n"
    for i, label1 in enumerate(labels):
        report += "{:>{}}".format(label1, column_width) + " ".join(["{:{}d}".format(cm[i, j], column_width) for j in range(len(labels))]) + "\n"
    return report

In [4]:
red = pd.read_csv('data/winequality-red1.csv')
white = pd.read_csv('data/winequality-white1.csv')

In [5]:
conditions = [(white['quality'] <= 5),
              (white['quality'] >= 7)]
choices = [0, 2]
white['quality_class'] = np.select(conditions, choices, default = 1)
conditions = [(red['quality'] <= 5),
              (red['quality'] >= 7)]
red['quality_class'] = np.select(conditions, choices, default = 1)

In [6]:
for column in white:
    if column != 'quality' and column != 'quality_class':
        mean = white[column].mean()
        sd = white[column].std()
        white[column] = (white[column]-mean)/sd
for column in red:
    if column != 'quality' and column != 'quality_class':
        mean = red[column].mean()
        sd = red[column].std()
        red[column] = (red[column]-mean)/sd

In [7]:
white_data = white.drop(['quality', 'quality_class'], axis = 1)
red_data = red.drop(['quality', 'quality_class'], axis = 1)

In [8]:
white_class = white['quality_class']
red_class = red['quality_class']

In [9]:
# Data and classes are prepared

print(white_data.describe())
print(white_class.describe())
print(red_data.describe())
print(red_class.describe())

In [10]:
white_data = white_data.values
white_class = white_class.values
#categorical_labels_white = to_categorical(white_class, num_classes=None)
red_data = red_data.values
red_class = red_class.values
#categorical_labels_red = to_categorical(red_class, num_classes=None)
number_of_features = 11

## Cross Validation

In [115]:
def rsme(targets, outputs):
    return tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(targets, outputs))))  

# Create function returning a compiled network
def create_network():
    network = Sequential()
    #network.add(Dense(32, input_dim=11))
    network.add(Dense(units=33, activation='sigmoid'))
    network.add(Dense(units=132, activation='sigmoid'))
    network.add(LeakyReLU(alpha=0.2))
    network.add(Dense(units=33))
    network.add(Dense(3, activation='softmax'))

    # network.compile(loss='binary_crossentropy', # Cross-entropy
    #                 optimizer='rmsprop', # Root Mean Square Propagation 'adam'
    #                metrics=['accuracy']) # Accuracy performance metric
    network.compile(loss=
                    tf.losses.sparse_softmax_cross_entropy(
                    labels=labels, weights=weights).eval(),#'sparse_categorical_crossentropy'
              optimizer='adam',
              metrics=['accuracy'])
    
    # Return compiled network
    return network

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [11]:
# Create function returning a compiled network
def create_network_2():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(33, activation=tf.nn.sigmoid),
        tf.keras.layers.Dense(132, activation=tf.nn.sigmoid),
        tf.keras.layers.Dense(132, activation=tf.nn.leaky_relu),
        tf.keras.layers.Dense(33),
        tf.keras.layers.Dense(3, activation=tf.nn.softmax)
    ])

    # network.compile(loss='binary_crossentropy', # Cross-entropy
    #                 optimizer='rmsprop', # Root Mean Square Propagation 'adam'
    #                metrics=['accuracy']) # Accuracy performance metric
    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    
    # Return compiled network
    return model

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [38]:
# Create function returning a compiled network
def create_network_3():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(33, activation=tf.nn.sigmoid),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(132, activation=tf.nn.sigmoid),
        tf.keras.layers.Dense(132*3, activation=tf.nn.leaky_relu),
        tf.keras.layers.Dense(312*3, activation=tf.nn.sigmoid),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(132*4, activation=tf.nn.selu),
        tf.keras.layers.Dense(312*16, activation=tf.nn.sigmoid),
        tf.keras.layers.Dense(132*16*4, activation=tf.nn.leaky_relu),
        tf.keras.layers.Dense(312*16, activation=tf.nn.sigmoid),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(33),
        tf.keras.layers.Dense(3, activation=tf.nn.softmax)
    ])

    # network.compile(loss='binary_crossentropy', # Cross-entropy
    #                 optimizer='rmsprop', # Root Mean Square Propagation 'adam'
    #                metrics=['accuracy']) # Accuracy performance metric
    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    
    # Return compiled network
    return model

cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [39]:
neural_network = KerasClassifier(build_fn=create_network_3, 
                                     epochs=1750,
                                     verbose=0)

## Red

In [40]:
prediction_red = cross_val_predict(neural_network, 
                                   red_data, red_class, cv=cv)

KeyboardInterrupt: 

In [ ]:
print(accuracy_score(red_class, prediction_red))
print(confusion_matrix_report(red_class, prediction_red))
print(classification_report(red_class, prediction_red))

In [24]:
print(math.sqrt(np.mean((red_class-prediction_red)*(red_class-prediction_red))))

0.6216844514890689


In [34]:
prediction_red_1000ep = prediction_red # accuracy 65%

In [73]:
prediction_red_500ep = prediction_red # accuracy 66%

In [19]:
prediction_red_1000ep_2 = prediction_red # accuracy 68%

In [41]:
print(math.sqrt(np.mean((red_class-prediction_red_1000ep_2)*(red_class-prediction_red_1000ep_2))))
print(accuracy_score(red_class, prediction_red_1000ep_2))
print(confusion_matrix_report(red_class, prediction_red_1000ep_2))
print(classification_report(red_class, prediction_red_1000ep_2))

0.6216844514890689
0.6791744840525328
      __Prediction___
         0     1     2
    0  524   200    20
    1  142   432    64
    2   15    72   130

              precision    recall  f1-score   support

           0       0.77      0.70      0.74       744
           1       0.61      0.68      0.64       638
           2       0.61      0.60      0.60       217

   micro avg       0.68      0.68      0.68      1599
   macro avg       0.66      0.66      0.66      1599
weighted avg       0.69      0.68      0.68      1599



## White

In [ ]:
prediction_white = cross_val_predict(neural_network, 
                                   white_data, white_class, cv=cv)

In [ ]:
print(accuracy_score(white_class, prediction_white))
print(confusion_matrix_report(white_class, prediction_white))
print(classification_report(white_class, prediction_white))

In [23]:
print(math.sqrt(np.mean((white_class-prediction_white)*(white_class-prediction_white))))

0.6193754675345038


In [25]:
prediction_white_1000ep = prediction_white # accuracy 69%

In [44]:
print(math.sqrt(np.mean((white_class-prediction_white_1000ep)*(white_class-prediction_white_1000ep))))
print(accuracy_score(white_class, prediction_white_1000ep))
print(confusion_matrix_report(white_class, prediction_white_1000ep))
print(classification_report(white_class, prediction_white_1000ep))

0.6193754675345038
0.6904859126173949
      __Prediction___
         0     1     2
    0 1153   430    57
    1  402  1548   248
    2   64   315   681

              precision    recall  f1-score   support

           0       0.71      0.70      0.71      1640
           1       0.68      0.70      0.69      2198
           2       0.69      0.64      0.67      1060

   micro avg       0.69      0.69      0.69      4898
   macro avg       0.69      0.68      0.69      4898
weighted avg       0.69      0.69      0.69      4898



In [ ]:
prediction_white_1000ep_deeper = prediction_white # accuracy 68.5%

# Krims Krams

model = tf.keras.Sequential()
model.add(layers.Dense(33, activation='sigmoid'))
model.add(layers.Dense(132, activation='sigmoid'))
model.add(layers.LeakyReLU(alpha=0.1))
model.add(layers.Dense(33, activation='linear'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer=tf.train.AdamOptimizer(), 
              loss='mean_squared_error',   # can also use sparse_categorical_crossentropy then you don't need labels
              metrics=['accuracy'])

white_sk_train, white_sk_test = train_test_split(white, test_size=0.2, random_state=42)
red_sk_train, red_sk_test = train_test_split(red, test_size=0.2, random_state=42)

white_data_train = white_sk_train.drop(['quality', 'quality_class'], axis = 1)
white_data_test = white_sk_test.drop(['quality', 'quality_class'], axis = 1)
red_data_train = red_sk_train.drop(['quality', 'quality_class'], axis = 1)
red_data_test = red_sk_test.drop(['quality', 'quality_class'], axis = 1)

white_class_train = white_sk_train['quality_class']
white_class_test = white_sk_test['quality_class']
red_class_train = red_sk_train['quality_class']
red_class_test = red_sk_test['quality_class']

In [30]:
model = tf.keras.Sequential()
model.add(layers.Dense(33, activation='sigmoid'))
model.add(layers.Dense(132, activation='sigmoid'))
model.add(layers.LeakyReLU(alpha=0.1))
model.add(layers.Dense(33, activation='linear'))
model.add(layers.Dense(10, activation='softmax'))

In [34]:
model.compile(optimizer=tf.train.AdamOptimizer(), 
              loss='mean_squared_error',   # can also use sparse_categorical_crossentropy then you don't need labels
              metrics=['accuracy'])

In [33]:
model.fit(white_data_train, white_class_train, epochs=100, batch_size=16)

In [ ]:
test_loss, test_acc = model.evaluate(white_data_test, white_class_test)
print('Test accuracy:', test_acc)

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(33, activation=tf.nn.relu),
  tf.keras.layers.Dense(132, activation=tf.nn.sigmoid),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(132*4, activation=tf.nn.selu),
  tf.keras.layers.Dense(132*16, activation=tf.nn.sigmoid),
  tf.keras.layers.Dense(132*4, activation=tf.nn.relu),
  tf.keras.layers.Dense(132, activation=tf.nn.sigmoid),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(33, activation=tf.nn.relu),
  tf.keras.layers.Dense(3, activation=tf.nn.softmax)
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(white_data_train, white_class_train, epochs=150)
model.evaluate(white_data_test, white_class_test)

In [31]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(33, activation=tf.nn.sigmoid),
    tf.keras.layers.Dense(132, activation=tf.nn.sigmoid),
    tf.keras.layers.Dense(132, activation=tf.nn.leaky_relu),
    tf.keras.layers.Dense(33),
    tf.keras.layers.Dense(3, activation=tf.nn.softmax)
])

model.epochs = 500
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

#model.fit(white_data_train, white_class_train, epochs=1000)
#model.evaluate(white_data_test, white_class_test)


ValueError: optimizer must be an instance of tf.train.Optimizer, not a <class 'str'>

white_data_train = white_data_train.values
white_class_train = white_class_train.values
categorical_labels_train = to_categorical(white_class_train, num_classes=None)
white_data_test = white_data_test.values
white_class_test = white_class_test.values
categorical_labels_test = to_categorical(white_class_test, num_classes=None)

In [ ]:
# Maximum Accuracy bisher 68% (very wide but not so deep network), konsistent 65%, new best is 0.706
# Could use weights for losses for balancing classes (as described by some paper or website)

In [ ]:
#predictions = model.predict(white_data_test)
predictions = model.predict_classes(white_data_test)

In [ ]:
print(accuracy_score(white_class_test, predictions))
print(confusion_matrix_report(white_class_test, predictions))
print(classification_report(white_class_test, predictions))